# Generate a Radial Bar Plot

# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/KiTH_Solutions/Research/Clinical Trial/study_metadata/all_performances.xlsx'

In [ ]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Dropbox (Partners HealthCare)/studies/review_pyper/figures/mansucript_figures/100_manuscript_questions'
sheet = 'study_results'

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()

# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
drop_list = ['ch_retro_antero']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
data_df.columns

Set the parameters for dropping rows

In [ ]:
column = 'Cohort'  # The column you'd like to evaluate
condition = 'equal'  # The condition to check ('equal', 'above', 'below')
value = 3  # The value to compare against

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['Age']

In [ ]:
data_df = cal_palm.standardize_columns(cols_not_to_standardize)
data_df

Descriptive Stats

In [ ]:
data_df.describe()

# 02 Plot

**Grouped Barplot**
- Expects a Dataframe with a category the grouping variable that sets colour. 
- variable represents each thig to be plotted, like 'neuroimaging, bias, etc'. 
- metric is the value of the variable to be plotted.

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

def plotly_grouped_radial_bar(dataframe, metric='Accuracy', category_col='Category', variable_col='Variable', title='Title Here'):
    """
    Creates a grouped radial bar chart with Plotly, where each 'category' is a group with a unique color,
    and each 'variable' within that category is an individual bar, evenly spaced around the plot.
    
    Args:
        dataframe (pandas.DataFrame): DataFrame containing the data to plot.
        metric (str): The metric to plot. Defaults to 'Accuracy'.
        category_col (str): The column name for the grouping category.
        variable_col (str): The column name for the individual variables.

    Returns:
        plotly.graph_objs._figure.Figure: Plotly Figure object for the radial bar chart.
    """
    # Assign colors to each category for grouping
    color_palette = px.colors.qualitative.T10
    category_list = dataframe[category_col].unique().tolist()
    color_dict = {category: color_palette[i % len(color_palette)] for i, category in enumerate(category_list)}

    # Determine the total number of unique variables across all categories
    total_variables = len(dataframe[variable_col].unique())

    # Initialize the figure
    fig = go.Figure()
    
    i = 0
    for category in category_list:
        category_data = dataframe[dataframe[category_col] == category]
        category_data.sort_values(by=[metric], inplace=True)
        fig.add_trace(go.Barpolar(
            r=[0],
            theta=[0],
            name=category,
            marker_color=color_dict[category],
            showlegend=True
            ))
        for j, (idx, row) in enumerate(category_data.iterrows()):
            # Calculate the angle for the current variable
            angle = (i * 360) / total_variables

            # Add a bar for the current variable
            fig.add_trace(go.Barpolar(
                r=[row[metric]],
                theta=[angle],
                width=[360 / total_variables],  # Slightly reduce width for spacing between bars
                name=f"{row[variable_col]}",
                marker_color=color_dict[row[category_col]],
                marker_line_color='black',
                marker_line_width=1,
                opacity=0.9,
                showlegend=False
            ))
            i = i+1

    # Handle Legend
    fig.update_layout(
        legend=dict(
            title=dict(text=variable_col),
            itemsizing='constant',
            orientation='h',
            traceorder='normal',
            x=0.5,  # Center the title
            xanchor='center',  # Ensure the title is centered
            yanchor='bottom',  # Position the title at the top of the plot
            font=dict(family="Myriad Pro", color='black', size=16)
            )
    )
    # Handle Title
    fig.update_layout(
    title=dict(
        text=title,  # Replace with your title
        x=0.5,  # Center the title
        xanchor='center',  # Ensure the title is centered
        yanchor='top',  # Position the title at the top of the plot
        font=dict(family="Myriad Pro", color='black', size=20)
        )
    )
    
    # Handle Colour
    fig.update_layout(
        polar=dict(
            bgcolor="white",  # Set the polar background color to white
            radialaxis=dict(showgrid=True, gridcolor='black'),  # Show radial axis grid lines
            angularaxis=dict(showgrid=False, gridcolor='gray', showticklabels=False)  # Show angular axis grid lines
        ),
        font=dict(
            family="Myriad Pro", 
            color='black'
        )
    )

    return fig

import os

def save_plot_as_svg(fig, out_dir, filename='radial_barplot.svg'):
    """
    Saves a Plotly figure as an SVG file in the specified directory.
    
    Args:
        fig (plotly.graph_objs._figure.Figure): The Plotly figure to save.
        out_dir (str): The directory path where the SVG file will be saved.
        filename (str): The filename for the SVG file. Defaults to 'plot.svg'.
    """
    # Ensure the output directory exists
    os.makedirs(out_dir, exist_ok=True)

    # Define the full path for the output file
    file_path = os.path.join(out_dir, filename)

    # Save the figure
    fig.write_image(file_path, format='svg', engine='kaleido')

    print(f"Saved figure to {file_path}")


In [ ]:
data_df.columns


In [ ]:
# Usage with your dataframe
fig = plotly_grouped_radial_bar(data_df, metric='Precision', category_col='Inference', variable_col='Variable', title='Accuracy of GPT Manuscript Read')
fig.show()

When you are happy with it, save it. 

In [ ]:
save_plot_as_svg(fig, out_dir,filename='radial_bar_plot_precision.svg' )

Enjoy
- Calvin